In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 

from model.mma_features import DataPreprocessor, SimpleFeatureExtractor, EloFeatureExtractor, BioFeatureExtractor

stats_df = pd.read_csv("data/clean_stats3.csv")
# run thru preprocessor to get _opp stats for each fight
DP = DataPreprocessor(stats_df)
pp_df = DP.get_preprocessed_df()
pp_df.head()

,Date,Opponent,Event,FighterResult,TSL,TSA,SSL,SSA,TSL-TSA,KD,...,SDHA_opp,SDLL_opp,SDLA_opp,TD_fails_opp,submission_rate_opp,distance_strikes_landed_opp,clinch_strikes_landed_opp,standing_strikes_opp,KD_power_opp,ground_strikes_landed_opp
0,1991-09-26,Murilo Bustamante,Desafio,L,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-01-01,Renzo Gracie,Desafio,L,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-29,Moura Moura,CP X CB,W,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-29,Jose Landi-Jons,CP X CB,L,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-11-08,Ken Shamrock,Pancrase,L,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
simple_fe = SimpleFeatureExtractor(pp_df)
simple_fe.fit_transform_all()
simple_fe.trans_df.head()

100%|██████████████████████████████████████████████████████████████████████| 4450/4450 [00:39<00:00, 113.93it/s]


,FighterID,OpponentID,Date,total_fights,total_ufc_fights,t_since_last_fight,total_fights_opp,total_ufc_fights_opp,t_since_last_fight_opp,t_since_last_fight_diff,t_since_last_fight_log_diff,total_fights_diff,total_fights_sqrt_diff,total_ufc_fights_diff,total_ufc_fights_sqrt_diff
0,2558095/marcelo-mendes,2354059/murilo-bustamante,1991-09-26,0,0,730,0,0,730,0,0.0,0,0.0,0,0.0
1,2501396/luiz-augusto-alvareda,2354119/renzo-gracie,1992-01-01,0,0,730,0,0,730,0,0.0,0,0.0,0,0.0
2,2354132/jose-landi-jons,3107994/moura-moura,1993-08-29,0,0,730,0,0,730,0,0.0,0,0.0,0,0.0
4,2557847/takaku-fuke,2335653/ken-shamrock,1993-11-08,0,0,730,0,0,730,0,0.0,0,0.0,0,0.0
8,2335738/gerard-gordeau,2504082/kevin-rosier,1993-11-12,0,0,730,0,0,730,0,0.0,0,0.0,0,0.0


In [4]:
# real_elo_target_cols = ["SSL", "TSL", "TDL"]
real_elo_target_cols = [
#     "fighter_result_time_left", 
    "ordinal_fighter_result",
    "submission_fighter_result",
    "tko_ko_fighter_result",
    "decision_fighter_result",
    "finish_fighter_result",
]
diff_elo_target_cols = [
    'TSL',
    'TDL',
    'TDS',
    'SSL',
    'SM',
    'RV',
    'KD',
    'SGHL',
    'SGBL',
    'SCBL',
    'SCHL',
    'ADTB',
    'ADTM',
    'AD',
    'TD_fails',
    'distance_strikes_landed',
    'clinch_strikes_landed',
    'standing_strikes',
    'ground_strikes_landed',
]
binary_elo_target_cols = ["Win"]

elo_fe = EloFeatureExtractor(pp_df.assign(Win=pp_df["FighterResult"].map({"W":1.,"L":0.,"D":np.nan})), 
                             elo_alpha=0.4,
                             real_elo_target_cols = real_elo_target_cols, 
                             diff_elo_target_cols = diff_elo_target_cols,
                             binary_elo_target_cols = binary_elo_target_cols)
elo_fe.fit_transform_all()
elo_fe.elo_df.head()

3959it [00:00, 6825.76it/s]
3959it [00:00, 7091.64it/s]
3959it [00:00, 7264.66it/s]
3962it [00:00, 6888.08it/s]
3959it [00:00, 7487.47it/s]
3959it [00:00, 6576.85it/s]
3959it [00:00, 6753.84it/s]
3959it [00:00, 6769.36it/s]
3959it [00:00, 6919.08it/s]
3959it [00:00, 7204.33it/s]
3962it [00:00, 7227.51it/s]
3959it [00:00, 7173.09it/s]
3959it [00:00, 6984.64it/s]
3959it [00:00, 7050.53it/s]
3962it [00:00, 7249.16it/s]
39390it [00:06, 6026.56it/s]
3959it [00:00, 6377.45it/s]
40119it [00:06, 6467.65it/s]
3962it [00:00, 6958.78it/s]
40119it [00:05, 7071.85it/s]
3959it [00:00, 6974.01it/s]
40119it [00:05, 6903.64it/s]
3959it [00:00, 7130.35it/s]
40119it [00:05, 6943.14it/s]
40119it [00:06, 6537.27it/s]


,FighterID,OpponentID,oldFighterEloAD,oldOpponentEloAD,predTargetAD,targetAD,Date,newFighterEloAD,newOpponentEloAD,oldEloDiffAD,...,newFighterElosubmission_fighter_result,newOpponentElosubmission_fighter_result,oldEloDiffsubmission_fighter_result,oldFighterElotko_ko_fighter_result,oldOpponentElotko_ko_fighter_result,predTargettko_ko_fighter_result,targettko_ko_fighter_result,newFighterElotko_ko_fighter_result,newOpponentElotko_ko_fighter_result,oldEloDifftko_ko_fighter_result
0,2335635/jason-delucia,2335757/trent-jenkins,0.0,0.0,0.0,1.000000,1993-11-12,0.20000,-0.20000,0.0,...,0.2000,-0.2000,0.000,0.0,0.0,0.0,0,0.0,0.0,0.0
1,2335738/gerard-gordeau,2504081/teila-tuli,0.0,0.0,0.0,0.000000,1993-11-12,0.00000,0.00000,0.0,...,-0.1280,-0.0320,-0.160,0.0,0.0,0.0,1,0.2,-0.2,0.0
2,2335728/zane-frazier,2504082/kevin-rosier,0.0,0.0,0.0,0.000000,1993-11-12,0.00000,0.00000,0.0,...,0.0000,0.0000,0.000,0.0,0.0,0.0,-1,-0.2,0.2,0.0
3,2335738/gerard-gordeau,2504082/kevin-rosier,0.0,0.0,0.0,0.000000,1993-11-12,0.00000,0.00000,0.0,...,-0.1024,-0.0256,-0.128,0.2,0.2,0.0,1,0.4,0.0,0.0
4,2335635/jason-delucia,2951141/scott-baker,0.2,0.0,0.2,0.267949,1994-03-11,0.21359,-0.01359,0.2,...,0.2256,-0.1936,0.032,0.0,0.0,0.0,0,0.0,0.0,0.0


In [5]:
feat_df = elo_fe.elo_df.merge(
    simple_fe.trans_df,
    on=["FighterID", "OpponentID", "Date"],
    how="inner"
)

bio_df = pd.read_csv("data/clean_bios.csv")
bio_fe = BioFeatureExtractor(bio_df)
feat_df = bio_fe.fit_transform_all(feat_df)
feat_df.head()

,FighterID,OpponentID,oldFighterEloAD,oldOpponentEloAD,predTargetAD,targetAD,Date,newFighterEloAD,newOpponentEloAD,oldEloDiffAD,...,imp_reach_opp,imp_weight_opp,imp_height_opp,age,age_opp,age_diff,reach_diff,weight_diff,log_weight_diff,height_diff
0,2335635/jason-delucia,2335757/trent-jenkins,0.0,0.0,0.0,1.000000,1993-11-12,0.20000,-0.20000,0.0,...,78.426958,250.000000,75.854283,NaN,NaN,0.0,-5.273051,-64.4375,-0.298069,-4.64658
1,2335738/gerard-gordeau,2504081/teila-tuli,0.0,0.0,0.0,0.000000,1993-11-12,0.00000,0.00000,0.0,...,78.426958,250.000000,75.854283,34.646575,NaN,0.0,0.000000,0.0000,0.000000,0.00000
2,2335728/zane-frazier,2504082/kevin-rosier,0.0,0.0,0.0,0.000000,1993-11-12,0.00000,0.00000,0.0,...,71.634058,167.003261,69.868379,27.345205,NaN,0.0,0.000000,0.0000,0.000000,0.00000
3,2335738/gerard-gordeau,2504082/kevin-rosier,0.0,0.0,0.0,0.000000,1993-11-12,0.00000,0.00000,0.0,...,71.634058,167.003261,69.868379,34.646575,NaN,0.0,0.000000,0.0000,0.000000,0.00000
4,2335635/jason-delucia,2951141/scott-baker,0.2,0.0,0.2,0.267949,1994-03-11,0.21359,-0.01359,0.2,...,78.426958,250.000000,75.854283,NaN,NaN,0.0,-5.273051,-64.4375,-0.298069,-4.64658


In [9]:
# okay i need the opening moneyline...
ml_df = pd.read_csv("data/clean_stats_plus_ml.csv")[[
    "Date", "FighterID", "OpponentID", 
    'FighterOpen', 'OpponentOpen',
    'FighterCloseLeft', 'FighterCloseRight', 'OpponentCloseLeft',
    'OpponentCloseRight',
]]
ml_df["Date"] = pd.to_datetime(ml_df["Date"])

def parse_american_odds(x:pd.Series):
    fav_inds = x <= 0
    dog_inds = x > 0
    y = pd.Series(0, index=x.index)
    y.loc[fav_inds] = -1 * x / (100 - x)
    y.loc[dog_inds] = 100 / (100 + x)
    return y

ml_df["p_fighter"] = parse_american_odds(ml_df["FighterOpen"])
ml_df["p_opponent"] = parse_american_odds(ml_df["OpponentOpen"])
ml_df["p_fighter_midpoint"] = (ml_df["p_fighter"] + 1 - ml_df["p_opponent"]) / 2
ml_df["p_fighter_implied"] = ml_df["p_fighter"] / (ml_df["p_fighter"] + ml_df["p_opponent"])
# print(feat_df.shape, ml_df.shape)
ml_df.columns

Index(['Date', 'FighterID', 'OpponentID', 'FighterOpen', 'OpponentOpen',
       'FighterCloseLeft', 'FighterCloseRight', 'OpponentCloseLeft',
       'OpponentCloseRight', 'p_fighter', 'p_opponent', 'p_fighter_midpoint',
       'p_fighter_implied'],
      dtype='object')

In [10]:
feat_ml_df = feat_df.merge(
    ml_df, 
    on=["Date", "FighterID", "OpponentID"],
    how="inner"
)
print(feat_ml_df.shape)
feat_ml_df.head()

(3220, 217)


,FighterID,OpponentID,oldFighterEloAD,oldOpponentEloAD,predTargetAD,targetAD,Date,newFighterEloAD,newOpponentEloAD,oldEloDiffAD,...,FighterOpen,OpponentOpen,FighterCloseLeft,FighterCloseRight,OpponentCloseLeft,OpponentCloseRight,p_fighter,p_opponent,p_fighter_midpoint,p_fighter_implied
0,2335629/bj-penn,2335885/jens-pulver,0.842355,0.364097,0.478257,2.645751,2007-06-23,1.275853,-0.069401,0.478257,...,-490.0,340.0,-357.0,-330.0,225.0,280.0,0.830508,0.227273,0.801618,0.785142
1,2335676/joe-lauzon,2354360/brandon-melendez,0.091024,0.000000,0.091024,2.000000,2007-06-23,0.472819,-0.381795,0.091024,...,-430.0,330.0,-700.0,-475.0,355.0,425.0,0.811321,0.232558,0.789381,0.777217
2,2335447/anderson-silva,2335475/nate-marquardt,0.227196,0.736307,-0.509111,-1.000000,2007-07-07,0.129018,0.834485,-0.509111,...,-170.0,150.0,-145.0,-145.0,125.0,125.0,0.629630,0.400000,0.614815,0.611511
3,2335302/heath-herring,2335521/antonio-rodrigo-nogueira,0.106410,0.000000,0.106410,-2.645751,2007-07-07,-0.444022,0.550432,0.106410,...,470.0,-810.0,500.0,500.0,-700.0,-700.0,0.175439,0.890110,0.142664,0.164646
4,2335694/frankie-edgar,2335717/mark-bocek,0.000000,0.000000,0.000000,1.414214,2007-07-07,0.282843,-0.282843,0.000000,...,-260.0,180.0,-230.0,-230.0,190.0,190.0,0.722222,0.357143,0.682540,0.669118


In [11]:
feat_cols = [
    "oldEloDiffordinal_fighter_result", 
    "oldEloDiffsubmission_fighter_result",
    "oldEloDifftko_ko_fighter_result",
    "oldEloDiffdecision_fighter_result",
#     "oldEloDifffinish_fighter_result",
#     "oldEloDifffighter_result_time_left",
    
#     'oldEloDiffAD', 
    'oldEloDiffADTB', 
    'oldEloDiffADTM', 
    'oldEloDiffKD',
    'oldEloDiffRV', 
#     'oldEloDiffSCBL', 
#     'oldEloDiffSCHL', 
    'oldEloDiffSGBL',
    'oldEloDiffSGHL', 
    'oldEloDiffSM', 'oldEloDiffSSL', 'oldEloDiffTDL',
    'oldEloDiffTDS', 
#     'oldEloDiffTD_fails', 
    'oldEloDiffTSL',
    'oldEloDiffclinch_strikes_landed', 
    'oldEloDiffdistance_strikes_landed',
    'oldEloDiffground_strikes_landed', 
    'oldEloDiffstanding_strikes',
    
    'oldEloDiffWin',
    
    "t_since_last_fight_log_diff", 
#     "t_since_last_fight_diff",
    "total_fights_sqrt_diff", 
    "total_ufc_fights_diff",
    
    "age_diff", "reach_diff", 
#     "weight_diff", 
    "log_weight_diff",
    "height_diff",
]

max_train_dt = pd.to_datetime("2021-07-01")

train_df = feat_ml_df.loc[feat_ml_df["Date"] <= max_train_dt].dropna(subset=[*feat_cols, "targetWin"])
test_df = feat_ml_df.loc[feat_ml_df["Date"] > max_train_dt].dropna(subset=[*feat_cols, "targetWin"])

In [12]:
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler(with_mean=False)
scale_ = (train_df[feat_cols]**2).mean(0)
X_train = train_df[feat_cols] / scale_
X_test = test_df[feat_cols] / scale_

y_train = train_df["targetWin"]
y_test = test_df["targetWin"]

In [13]:
def logit(x):
    return np.log(x) - np.log(1-x)

ml_train = logit(train_df["p_fighter_implied"])
ml_test = logit(test_df["p_fighter_implied"])

In [14]:
# okay uhh
import stan

code = """

data {
    int<lower=0> n;                     // number of data points in training data
    int<lower=0> n2;                    // number of data points in test data
    int<lower=1> d;                     // explanatory variable dimension
    int<lower=0,upper=1> y[n];          // response variable
    real<lower=0> beta_prior_std;       // prior scale on beta

    matrix[n, d] X;                     // explanatory variable
    vector[n] ml_logit;                   // logit of the opening money line

    matrix[n2, d] X2;                   // test data
    vector[n2] ml_logit2;                 // test data

}

parameters {
    vector[d] beta;
}

transformed parameters {
    vector[n] eta;
    vector[n2] eta2;
    eta = ml_logit + (X * beta);      // linear predictor
    eta2 = ml_logit2 + (X2 * beta);   // linear predictor for test data
}

model {
    for(i in 1:d){
        beta[i] ~ normal(0, beta_prior_std);
        //beta[i] ~ cauchy(0, beta_prior_std); //prior for slopes following gelman 2008
    }

    // observation model
    y ~ bernoulli_logit(eta);
}

generated quantities {
    vector[n2] y_pred;
    y_pred = inv_logit(eta2);  // y values predicted for test data
}
"""

class SimpleSymmetricModel(object):
    
    def __init__(self, beta_prior_std=0.1, num_chains=4, num_samples=1000):
        self.beta_prior_std = float(beta_prior_std)
        self.code = code
        self.scale_ = None
        self.fit = None
        self.num_chains = num_chains
        self.num_samples = num_samples
        
    def _fit(self, data):
        posterior = stan.build(self.code, data=data, random_seed=1)
        fit = posterior.sample(num_chains=self.num_chains, num_samples=self.num_samples)
        self.fit = fit
        return fit
        
    def fit_predict(self, train_df, test_df, feat_cols):
        scale_ = (train_df[feat_cols]**2).mean(0)
        self.scale_ = scale_
        X_train = train_df[feat_cols] / scale_
        X_test = test_df[feat_cols] / scale_

        y_train = train_df["targetWin"]
        y_test = test_df["targetWin"]

        ml_train = logit(train_df["p_fighter_implied"])
        ml_test = logit(test_df["p_fighter_implied"])
        
        data = {
            "n": train_df.shape[0],
            "n2": test_df.shape[0],
            "d": len(feat_cols),
            "y": y_train.astype(int).values,
            "beta_prior_std": self.beta_prior_std,
            "X": X_train.values,
            "ml_logit": ml_train.values,
            "X2": X_test.values,
            "ml_logit2": ml_test.values,
        }

        fit = self._fit(data)
        return fit["y_pred"].mean(1)

sym_model = SimpleSymmetricModel(beta_prior_std=2.0, num_samples=500)
y_pred = sym_model.fit_predict(train_df, test_df, feat_cols)
(y_pred.round() == test_df["targetWin"]).mean()

Building: 0.6s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s


In file included from /Users/johncurcio/Library/Caches/httpstan/4.7.2/models/hwcu7qli/model_hwcu7qli.cpp:2:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:8:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math/rev/core.hpp:28:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math/rev/core/operator_addition.hpp:6:
/usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math/prim/err/check_matching_dims.hpp:33:8: warning: unused variable 'error' [-Wunus

Building: 3.7s
Building: 3.8s
Building: 4.0s


In file included from /Users/johncurcio/Library/Caches/httpstan/4.7.2/models/hwcu7qli/model_hwcu7qli.cpp:2:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:8:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math/rev/core.hpp:53:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math/rev/core/profiling.hpp:9:
In file included from /usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/include/stan/math/prim/err.hpp:50:
/usr/local/Caskroom/miniconda/base/envs/sports/l

Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s


...

63 warnings generated.


Building: 17.3s
Building: 17.4s
Building: 17.5s
Building: 17.6s
Building: 17.7s
Building: 17.8s


ld: warning: direct access in function 'long double boost::math::detail::bessel_j0<long double>(long double)' from file '/usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/stan_services.o' to global weak symbol 'long double boost::math::detail::bessel_j0<long double>(long double)::Q2' from file 'build/temp.macosx-10.9-x86_64-3.9/Users/johncurcio/Library/Caches/httpstan/4.7.2/models/hwcu7qli/model_hwcu7qli.o' means the weak symbol cannot be overridden at runtime. This was likely caused by different translation units being compiled with different visibility settings.
ld: warning: direct access in function 'long double boost::math::detail::bessel_j0<long double>(long double)' from file '/usr/local/Caskroom/miniconda/base/envs/sports/lib/python3.9/site-packages/httpstan/stan_services.o' to global weak symbol 'long double boost::math::detail::bessel_j0<long double>(long double)::P2' from file 'build/temp.macosx-10.9-x86_64-3.9/Users/johncurcio/Library/Caches

Messages from stanc:
Warning in '/var/folders/rv/xzq99st968x3_ylm6n80zn4c0000gq/T/httpstan_u7slrv3y/model_hwcu7qli.stan', line 7, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Sampling:   0%
Sampling:   0% (1/6000)
Sampling:   0% (2/6000)
Sampling:   0% (3/6000)
Sampling:   0% (4/6000)
Sampling:   2% (103/6000)
Sampling:   3% (202/6000)
Sampling:   5% (301/6000)
Sampling:   7% (400/6000)
Sampling:   8% (500/6000)
Sampling:  10% (600/6000)
Sampling:  12% (700/6000)
Sampling:  13% (800/6000)
Sampling:  15% (900/6000)
Sampling:  17% (1000/6000)
Sampling:  18% (1100/6000)
Sampling:  20% (1200/6000)
Sampling:  22% (1300/6000)
Sampling:  23% (1400/6000)
Sampling:  25% (1500/6000)
Sampling:  27% (1600/6000)
Sampling:  28% (1700/6000)
Sampling:  30% (1800/6000)
Sampling:  32% (1900/6000

0.6525679758308157

In [15]:
from sklearn.metrics import log_loss

log_loss(y_pred=y_pred, y_true=y_test)

0.6099569250376933

In [16]:
pd.DataFrame(list(zip(feat_cols, sym_model.fit["beta"].mean(1)))).sort_values(1)

,0,1
0,oldEloDiffordinal_fighter_result,-1.520902
22,total_ufc_fights_diff,-0.418321
17,oldEloDiffground_strikes_landed,-0.172423
14,oldEloDiffTSL,-0.065710
26,height_diff,-0.045838
20,t_since_last_fight_log_diff,-0.036273
6,oldEloDiffKD,-0.018428
5,oldEloDiffADTM,-0.009286
7,oldEloDiffRV,-0.001094
19,oldEloDiffWin,-0.000615


In [17]:
from sklearn.decomposition import PCA, KernelPCA

class PcaSymmetricModel(object):
    
    def __init__(self, beta_prior_std=0.1, n_pca=8, num_chains=4, num_samples=1000):
        self.beta_prior_std = float(beta_prior_std)
        self.n_pca = n_pca
        self.code = code
        self.scale_ = None
        self.pca = PCA(n_components=n_pca, whiten=True)
        self.fit = None
        self.num_chains = num_chains
        self.num_samples = num_samples
        
    def _fit(self, data):
        posterior = stan.build(self.code, data=data, random_seed=1)
        fit = posterior.sample(num_chains=self.num_chains, num_samples=self.num_samples)
        self.fit = fit
        return fit
        
    def fit_predict(self, train_df, test_df, feat_cols):
        scale_ = (train_df[feat_cols]**2).mean(0)
        self.scale_ = scale_
        X_train = train_df[feat_cols] / scale_
        X_test = test_df[feat_cols] / scale_
        
        # pca happens here
        X_pca_train = self.pca.fit_transform(X_train)
        X_pca_test = self.pca.transform(X_test)

        y_train = train_df["targetWin"]
        y_test = test_df["targetWin"]

        ml_train = logit(train_df["p_fighter_implied"])
        ml_test = logit(test_df["p_fighter_implied"])
        
        data = {
            "n": train_df.shape[0],
            "n2": test_df.shape[0],
            "d": X_pca_train.shape[1],
            "y": y_train.astype(int).values,
            "beta_prior_std": self.beta_prior_std,
            "X": X_pca_train,
            "ml_logit": ml_train.values,
            "X2": X_pca_test,
            "ml_logit2": ml_test.values,
        }

        fit = self._fit(data)
        return fit["y_pred"].mean(1)

pca_model = PcaSymmetricModel(n_pca=16, beta_prior_std=1.0, num_samples=500)
y_pred = pca_model.fit_predict(train_df, test_df, feat_cols)
acc = (y_pred.round() == test_df["targetWin"]).mean()
print(f"accuracy: {acc}")
print(f"log loss: {log_loss(y_pred=y_pred, y_true=y_test)}")

Building: found in cache, done.
Messages from stanc:
Warning in '/var/folders/rv/xzq99st968x3_ylm6n80zn4c0000gq/T/httpstan_u7slrv3y/model_hwcu7qli.stan', line 7, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Sampling:   0%
Sampling:   7% (400/6000)
Sampling:  13% (800/6000)
Sampling:  18% (1100/6000)
Sampling:  22% (1300/6000)
Sampling:  28% (1700/6000)
Sampling:  35% (2100/6000)
Sampling:  55% (3300/6000)
Sampling:  77% (4600/6000)
Sampling:  88% (5300/6000)
Sampling: 100% (6000/6000)
Sampling: 100% (6000/6000), done.
Messages received during sampling:
  Gradient evaluation took 0.000146 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.46 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.000318 seconds
  1000 transitions

accuracy: 0.649546827794562
log loss: 0.6090773076393765


In [18]:
gender_df = pd.read_csv("data/fighter_genders.csv")
# gender_map = gender_df.set_index("FighterID").to_dict()["gender"]

train_gender_df = train_df.merge(gender_df, on="FighterID", how="left")
test_gender_df = test_df.merge(gender_df, on="FighterID", how="left")

train_gender_df["gender"].isnull().any(), test_gender_df["gender"].isnull().any()

(False, False)

In [19]:
# okay uhh
import stan

hier_code = """

data {
    int<lower=0> n;                     // number of data points in training data
    int<lower=0> n2;                    // number of data points in test data
    int<lower=1> d;                     // explanatory variable dimension
    int<lower=0,upper=1> y[n];          // response variable
    real<lower=0> beta_prior_std;       // prior scale on beta mean across groups
    real<lower=0> intra_group_std;      // prior scale on beta, std dev of group's beta around mean
    
    int<lower=1> L;                     // number of levels (if this is gender, then it's just 2)
    int<lower=1,upper=L> ll[n];         // level of each fight in train
    int<lower=1,upper=L> ll2[n2];       // level of each fight in test

    matrix[n, d] X;                     // explanatory variable
    vector[n] ml_logit;                   // logit of the opening money line

    matrix[n2, d] X2;                   // test data
    vector[n2] ml_logit2;                 // test data

}

parameters {
    vector[d] beta[L];    // coefficient values for each level
    real mu[d];           // mean of coefficient values for each level
}

transformed parameters {
    //vector[n] eta;
    //vector[n2] eta2;
    //eta = ml_logit + (X * beta);      // linear predictor
    //eta2 = ml_logit2 + (X2 * beta);   // linear predictor for test data
}

model {

    mu ~ normal(0, beta_prior_std);
    for (l in 1:L)
        beta[l] ~ normal(mu, intra_group_std); // TODO magic number - more or less defines covariance btw genders

    vector[n] x_beta_ll;
    for (i in 1:n)
        x_beta_ll[i] = X[i] * beta[ll[i]];
    y ~ bernoulli_logit(ml_logit + x_beta_ll);
}

generated quantities {
    vector[n2] y_pred;
    
    for(i in 1:n2){
        y_pred[i] = inv_logit(ml_logit2[i] + X[i] * beta[ll2[i]]);
    }
    //y_pred = inv_logit(eta2);  // y values predicted for test data
}
"""

class HierarchicalSymmetricModel(SimpleSymmetricModel):
    
    def __init__(self, beta_prior_std=1.0, intra_group_std=0.1, num_chains=4, num_samples=100):
        self.beta_prior_std = float(beta_prior_std)
        self.intra_group_std = float(intra_group_std)
        self.code = hier_code
        self.scale_ = None
        self.fit = None
        self.num_chains = num_chains
        self.num_samples = num_samples
        
    def fit_predict(self, train_df, test_df, feat_cols):
        scale_ = (train_df[feat_cols]**2).mean(0)
        self.scale_ = scale_
        X_train = train_df[feat_cols] / scale_
        X_test = test_df[feat_cols] / scale_

        y_train = train_df["targetWin"]
        y_test = test_df["targetWin"]

        ml_train = logit(train_df["p_fighter_implied"])
        ml_test = logit(test_df["p_fighter_implied"])
        
        gender_int_train = train_df["gender"].map({"M":1, "W":2})
        gender_int_test = test_df["gender"].map({"M":1, "W":2})
        
        data = {
            "n": train_df.shape[0],
            "n2": test_df.shape[0],
            "d": len(feat_cols),
            "y": y_train.astype(int).values,
            "beta_prior_std": self.beta_prior_std,
            "intra_group_std": self.intra_group_std,
            "L": 2,
            "ll": gender_int_train.values,
            "ll2": gender_int_test.values,
            "X": X_train.values,
            "ml_logit": ml_train.values,
            "X2": X_test.values,
            "ml_logit2": ml_test.values,
        }
        fit = self._fit(data)
        return fit["y_pred"].mean(1)

hier_model = HierarchicalSymmetricModel(beta_prior_std=1.0, intra_group_std=0.5)
y_pred = hier_model.fit_predict(train_gender_df, test_gender_df, feat_cols)
(y_pred.round() == test_gender_df["targetWin"]).mean()

Building: 0.2s
Building: 0.3s
Building: 0.5s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.3s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s
Building: 6.5s
Building: 6.6s
Building: 6.7s
Building: 6.8s
Building: 6.9s
Building: 7.0s
Building: 

Building: 26.4s
Building: 26.5s
Building: 26.7s
Building: 26.8s
Building: 26.9s
Building: 27.0s
Building: 27.1s
Building: 27.2s
Building: 27.3s
Building: 27.4s
Building: 27.5s
Building: 27.6s
Building: 27.7s
Building: 27.8s
Building: 27.9s
Building: 28.0s
Building: 28.1s
Building: 28.2s
Building: 28.3s
Building: 28.4s
Building: 28.5s
Building: 28.6s
Building: 28.7s
Building: 28.9s
Building: 29.0s
Building: 29.1s
Building: 29.2s
Building: 29.3s
Building: 29.4s
Building: 29.5s
Building: 29.6s
Building: 29.7s
Building: 29.8s
Building: 29.9s
Building: 30.0s
Building: 30.1s
Building: 30.2s
Building: 30.3s
Building: 30.4s
Building: 30.5s
Building: 30.6s
Building: 30.7s
Building: 30.8s
Building: 30.9s
Building: 31.0s
Building: 31.2s
Building: 31.3s
Building: 31.4s
Building: 31.5s
Building: 31.6s
Building: 31.7s
Building: 31.8s
Building: 31.9s
Building: 32.0s
Building: 32.1s
Building: 32.2s
Building: 32.3s
Building: 32.4s
Building: 32.5s
Building: 32.6s
Building: 32.7s
Building: 32.8s
Building

Building: 52.3s
Building: 52.4s
Building: 52.5s
Building: 52.6s
Building: 52.7s
Building: 52.9s
Building: 53.0s
Building: 53.1s
Building: 53.2s
Building: 53.3s
Building: 53.4s
Building: 53.5s
Building: 53.6s
Building: 53.7s
Building: 53.8s
Building: 53.9s
Building: 54.0s
Building: 54.1s
Building: 54.2s
Building: 54.3s
Building: 54.4s
Building: 54.5s
Building: 54.6s
Building: 54.7s
Building: 54.8s
Building: 54.9s
Building: 55.0s
Building: 55.1s
Building: 55.2s
Building: 55.3s
Building: 55.4s
Building: 55.5s
Building: 55.7s
Building: 55.8s
Building: 55.9s
Building: 56.0s
Building: 56.1s
Building: 56.2s
Building: 56.3s
Building: 56.4s
Building: 56.5s
Building: 56.6s
Building: 56.7s
Building: 56.8s
Building: 56.9s
Building: 57.0s
Building: 57.1s
Building: 57.2s
Building: 57.3s
Building: 57.4s
Building: 57.5s
Building: 57.6s
Building: 57.8s
Building: 57.9s
Building: 58.0s
Building: 58.1s
Building: 58.2s
Building: 58.3s
Building: 58.4s
Building: 58.5s
Building: 58.6s
Building: 58.7s
Building

Building: 78.2s
Building: 78.3s
Building: 78.4s
Building: 78.5s
Building: 78.6s
Building: 78.7s
Building: 78.8s
Building: 78.9s
Building: 79.1s
Building: 79.2s
Building: 79.3s
Building: 79.4s
Building: 79.5s
Building: 79.6s
Building: 79.7s
Building: 79.8s
Building: 79.9s
Building: 80.0s
Building: 80.1s
Building: 80.2s
Building: 80.3s
Building: 80.4s
Building: 80.5s
Building: 80.6s
Building: 80.7s
Building: 80.8s
Building: 80.9s
Building: 81.0s
Building: 81.1s
Building: 81.3s
Building: 81.4s
Building: 81.5s
Building: 81.6s
Building: 81.7s
Building: 81.8s
Building: 81.9s
Building: 82.0s
Building: 82.1s
Building: 82.2s
Building: 82.3s
Building: 82.4s
Building: 82.5s
Building: 82.6s
Building: 82.7s
Building: 82.8s
Building: 82.9s
Building: 83.0s
Building: 83.1s
Building: 83.2s
Building: 83.3s
Building: 83.4s
Building: 83.6s
Building: 83.7s
Building: 83.8s
Building: 83.9s
Building: 84.0s
Building: 84.1s
Building: 84.2s
Building: 84.3s
Building: 84.4s
Building: 84.5s
Building: 84.6s
Building

Building: 103.9s
Building: 104.0s
Building: 104.1s
Building: 104.3s
Building: 104.4s
Building: 104.5s
Building: 104.6s
Building: 104.7s
Building: 104.8s
Building: 104.9s
Building: 105.0s
Building: 105.1s
Building: 105.2s
Building: 105.3s
Building: 105.4s
Building: 105.5s
Building: 105.6s
Building: 105.7s
Building: 105.8s
Building: 105.9s
Building: 106.0s
Building: 106.1s
Building: 106.2s
Building: 106.3s
Building: 106.4s
Building: 106.5s
Building: 106.6s
Building: 106.7s
Building: 106.8s
Building: 106.9s
Building: 107.0s
Building: 107.1s
Building: 107.2s
Building: 107.3s
Building: 107.5s
Building: 107.6s
Building: 107.7s
Building: 107.8s
Building: 107.9s
Building: 108.0s
Building: 108.1s
Building: 108.2s
Building: 108.3s
Building: 108.4s
Building: 108.5s
Building: 108.6s
Building: 108.7s
Building: 108.8s
Building: 108.9s
Building: 109.0s
Building: 109.1s
Building: 109.2s
Building: 109.3s
Building: 109.4s
Building: 109.5s
Building: 109.7s
Building: 109.8s
Building: 109.9s
Building: 110.

...

Building: 204.2s
Building: 204.3s
Building: 204.4s
Building: 204.5s
Building: 204.6s
Building: 204.8s
Building: 204.9s
Building: 205.0s
Building: 205.1s
Building: 205.2s
Building: 205.3s
Building: 205.4s
Building: 205.5s
Building: 205.6s
Building: 205.7s
Building: 205.8s
Building: 205.9s
Building: 206.0s
Building: 206.1s
Building: 206.2s
Building: 206.3s
Building: 206.4s
Building: 206.5s
Building: 206.6s
Building: 206.7s
Building: 206.8s
Building: 207.0s
Building: 207.1s
Building: 207.2s
Building: 207.3s
Building: 207.4s
Building: 207.5s
Building: 207.6s
Building: 207.7s
Building: 207.8s
Building: 207.9s
Building: 208.0s
Building: 208.1s
Building: 208.2s
Building: 208.3s
Building: 208.4s
Building: 208.5s
Building: 208.6s
Building: 208.7s
Building: 208.8s
Building: 208.9s
Building: 209.0s
Building: 209.1s
Building: 209.3s
Building: 209.4s
Building: 209.5s
Building: 209.6s
Building: 209.7s
Building: 209.8s
Building: 209.9s
Building: 210.0s
Building: 210.1s
Building: 210.2s
Building: 210.

Building: 229.3s
Building: 229.4s
Building: 229.5s
Building: 229.6s
Building: 229.7s
Building: 229.8s
Building: 229.9s
Building: 230.1s
Building: 230.2s
Building: 230.3s
Building: 230.4s
Building: 230.5s
Building: 230.6s
Building: 230.7s
Building: 230.8s
Building: 230.9s
Building: 231.0s
Building: 231.1s
Building: 231.2s
Building: 231.3s
Building: 231.4s
Building: 231.5s
Building: 231.6s
Building: 231.7s
Building: 231.8s
Building: 231.9s
Building: 232.0s
Building: 232.1s
Building: 232.2s
Building: 232.3s
Building: 232.4s
Building: 232.5s
Building: 232.6s
Building: 232.7s
Building: 232.8s
Building: 232.9s
Building: 233.0s
Building: 233.2s
Building: 233.3s
Building: 233.4s
Building: 233.5s
Building: 233.6s
Building: 233.7s
Building: 233.8s
Building: 233.9s
Building: 234.0s
Building: 234.1s
Building: 234.2s
Building: 234.3s
Building: 234.4s
Building: 234.5s
Building: 234.6s
Building: 234.7s
Building: 234.8s
Building: 234.9s
Building: 235.0s
Building: 235.1s
Building: 235.2s
Building: 235.

Building: 254.4s
Building: 254.5s
Building: 254.6s
Building: 254.7s
Building: 254.8s
Building: 254.9s
Building: 255.0s
Building: 255.1s
Building: 255.2s
Building: 255.3s
Building: 255.4s
Building: 255.5s
Building: 255.6s
Building: 255.7s
Building: 255.8s
Building: 255.9s
Building: 256.0s
Building: 256.1s
Building: 256.2s
Building: 256.3s
Building: 256.4s
Building: 256.6s
Building: 256.7s
Building: 256.8s
Building: 256.9s
Building: 257.0s
Building: 257.1s
Building: 257.2s
Building: 257.3s
Building: 257.4s
Building: 257.5s
Building: 257.6s
Building: 257.7s
Building: 257.8s
Building: 257.9s
Building: 258.0s
Building: 258.1s
Building: 258.2s
Building: 258.3s
Building: 258.4s
Building: 258.5s
Building: 258.6s
Building: 258.7s
Building: 258.9s
Building: 259.0s
Building: 259.1s
Building: 259.2s
Building: 259.3s
Building: 259.4s
Building: 259.5s
Building: 259.6s
Building: 259.7s
Building: 259.8s
Building: 259.9s
Building: 260.0s
Building: 260.1s
Building: 260.2s
Building: 260.3s
Building: 260.

Building: 279.5s
Building: 279.6s
Building: 279.7s
Building: 279.8s
Building: 279.9s
Building: 280.0s
Building: 280.1s
Building: 280.2s
Building: 280.3s
Building: 280.4s
Building: 280.5s
Building: 280.6s
Building: 280.7s
Building: 280.9s
Building: 281.0s
Building: 281.1s
Building: 281.2s
Building: 281.3s
Building: 281.4s
Building: 281.5s
Building: 281.6s
Building: 281.7s
Building: 281.8s
Building: 281.9s
Building: 282.0s
Building: 282.1s
Building: 282.2s
Building: 282.3s
Building: 282.4s
Building: 282.5s
Building: 282.6s
Building: 282.7s
Building: 282.8s
Building: 282.9s
Building: 283.0s
Building: 283.1s
Building: 283.2s
Building: 283.3s
Building: 283.4s
Building: 283.5s
Building: 283.6s
Building: 283.7s
Building: 283.8s
Building: 283.9s
Building: 284.0s
Building: 284.1s
Building: 284.2s
Building: 284.3s
Building: 284.4s
Building: 284.6s
Building: 284.7s
Building: 284.8s
Building: 284.9s
Building: 285.0s
Building: 285.1s
Building: 285.2s
Building: 285.3s
Building: 285.4s
Building: 285.

TimeoutError: 

In [ ]:
# since i chose an aggressively tight spread btw m and w, 
# this should be close to 0.6087682255527661
log_loss(y_pred=y_pred, y_true=y_test)

In [ ]:
m_beta = hier_model.fit["beta"][0].mean(1)
w_beta = hier_model.fit["beta"][1].mean(1)

beta_df = pd.DataFrame({
    "m_beta": m_beta,
    "w_beta": w_beta,
    "diff": m_beta - w_beta,
    "feat": feat_cols,
})
beta_df.sort_values("diff")

# Trying some weird indexing

In [20]:
# okay uhh
import stan

hier_code = """

data {
    int<lower=0> n;                     // number of data points in training data
    int<lower=0> n2;                    // number of data points in test data
    int<lower=1> d;                     // explanatory variable dimension
    int<lower=0,upper=1> y[n];          // response variable
    real<lower=0> beta_prior_std;       // prior scale on beta mean across groups
    real<lower=0> intra_group_std;      // prior scale on beta, std dev of group's beta around mean
    
    vector[n] is_m;      // 0 if woman, 1 if man
    vector[n2] is_m2;    // 0 if woman, 1 if man
    
    matrix[n, d] X;                     // explanatory variable
    vector[n] ml_logit;                   // logit of the opening money line

    matrix[n2, d] X2;                   // test data
    vector[n2] ml_logit2;                 // test data

}

parameters {
    vector[d] beta_m;
    vector[d] beta_w;
}

transformed parameters {
    vector[n] eta;
    vector[n2] eta2;
    eta = (
        ml_logit + 
        ((X * beta_m) .* is_m) + 
        ((X * beta_w) .* (1 - is_m))
    );      // linear predictor
    eta2 = (
        ml_logit2 + 
        ((X2 * beta_m) .* is_m2) + 
        ((X2 * beta_w) .* (1 - is_m2))
    );   // linear predictor for test data
}

model {
    beta_m ~ normal(0, beta_prior_std);
    beta_w ~ normal(beta_m, intra_group_std); // damn i hope this works

    y ~ bernoulli_logit(eta);
}

generated quantities {
    vector[n2] y_pred;
    
    y_pred = inv_logit(eta2);  // y values predicted for test data
}
"""

class HierarchicalSymmetricModel(SimpleSymmetricModel):
    
    def __init__(self, beta_prior_std=1.0, intra_group_std=0.1, num_chains=4, num_samples=100):
        self.beta_prior_std = float(beta_prior_std)
        self.intra_group_std = float(intra_group_std)
        self.code = hier_code
        self.scale_ = None
        self.fit = None
        self.num_chains = num_chains
        self.num_samples = num_samples
        
    def fit_predict(self, train_df, test_df, feat_cols):
        scale_ = (train_df[feat_cols]**2).mean(0)
        self.scale_ = scale_
        X_train = train_df[feat_cols] / scale_
        X_test = test_df[feat_cols] / scale_

        y_train = train_df["targetWin"]
        y_test = test_df["targetWin"]

        ml_train = logit(train_df["p_fighter_implied"])
        ml_test = logit(test_df["p_fighter_implied"])
        
        is_m_train = train_df["gender"].map({"M":1, "W":0})
        is_m_test = test_df["gender"].map({"M":1, "W":0})
        
        data = {
            "n": train_df.shape[0],
            "n2": test_df.shape[0],
            "d": len(feat_cols),
            "y": y_train.astype(int).values,
            "beta_prior_std": self.beta_prior_std,
            "intra_group_std": self.intra_group_std,
            "is_m": is_m_train.values,
            "is_m2": is_m_test.values,
            "X": X_train.values,
            "ml_logit": ml_train.values,
            "X2": X_test.values,
            "ml_logit2": ml_test.values,
        }
        fit = self._fit(data)
        return fit["y_pred"].mean(1)

hier_model = HierarchicalSymmetricModel(beta_prior_std=1.0, intra_group_std=0.1, num_samples=500)
y_pred = hier_model.fit_predict(train_gender_df, test_gender_df, feat_cols)
acc = (y_pred.round() == test_df["targetWin"]).mean()
print(f"accuracy: {acc}")
print(f"log loss: {log_loss(y_pred=y_pred, y_true=y_test)}")

Building: 0.2s
Building: 0.3s
Building: 0.5s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.3s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s
Building: 6.5s
Building: 6.6s
Building: 6.7s
Building: 6.8s
Building: 6.9s
Building: 7.0s
Building: 

Building: 26.4s
Building: 26.6s
Building: 26.7s
Building: 26.8s
Building: 26.9s
Building: 27.0s
Building: 27.1s
Building: 27.2s
Building: 27.3s
Building: 27.4s
Building: 27.5s
Building: 27.6s
Building: 27.7s
Building: 27.8s
Building: 27.9s
Building: 28.0s
Building: 28.1s
Building: 28.2s
Building: 28.3s
Building: 28.4s
Building: 28.5s
Building: 28.6s
Building: 28.7s
Building: 28.8s
Building: 28.9s
Building: 29.0s
Building: 29.1s
Building: 29.2s
Building: 29.3s
Building: 29.4s
Building: 29.6s
Building: 29.7s
Building: 29.8s
Building: 29.9s
Building: 30.0s
Building: 30.1s
Building: 30.2s
Building: 30.3s
Building: 30.4s
Building: 30.5s
Building: 30.6s
Building: 30.7s
Building: 30.8s
Building: 30.9s
Building: 31.0s
Building: 31.1s
Building: 31.2s
Building: 31.3s
Building: 31.4s
Building: 31.5s
Building: 31.7s
Building: 31.8s
Building: 31.9s
Building: 32.0s
Building: 32.1s
Building: 32.2s
Building: 32.3s
Building: 32.4s
Building: 32.5s
Building: 32.6s
Building: 32.7s
Building: 32.8s
Building

Building: 52.4s
Building: 52.5s
Building: 52.6s
Building: 52.7s
Building: 52.8s
Building: 52.9s
Building: 53.0s
Building: 53.1s
Building: 53.2s
Building: 53.3s
Building: 53.4s
Building: 53.6s
Building: 53.7s
Building: 53.8s
Building: 53.9s
Building: 54.0s
Building: 54.1s
Building: 54.2s
Building: 54.3s
Building: 54.4s
Building: 54.5s
Building: 54.6s
Building: 54.7s
Building: 54.8s
Building: 54.9s
Building: 55.0s
Building: 55.1s
Building: 55.2s
Building: 55.3s
Building: 55.4s
Building: 55.5s
Building: 55.6s
Building: 55.7s
Building: 55.8s
Building: 55.9s
Building: 56.0s
Building: 56.1s
Building: 56.2s
Building: 56.3s
Building: 56.4s
Building: 56.5s
Building: 56.7s
Building: 56.8s
Building: 56.9s
Building: 57.0s
Building: 57.1s
Building: 57.2s
Building: 57.3s
Building: 57.4s
Building: 57.5s
Building: 57.6s
Building: 57.7s
Building: 57.8s
Building: 57.9s
Building: 58.0s
Building: 58.1s
Building: 58.2s
Building: 58.3s
Building: 58.4s
Building: 58.5s
Building: 58.6s
Building: 58.7s
Building

Building: 78.3s
Building: 78.4s
Building: 78.5s
Building: 78.6s
Building: 78.7s
Building: 78.8s
Building: 78.9s
Building: 79.0s
Building: 79.1s
Building: 79.2s
Building: 79.3s
Building: 79.4s
Building: 79.5s
Building: 79.6s
Building: 79.7s
Building: 79.8s
Building: 79.9s
Building: 80.0s
Building: 80.1s
Building: 80.2s
Building: 80.3s
Building: 80.4s
Building: 80.5s
Building: 80.6s
Building: 80.7s
Building: 80.8s
Building: 80.9s
Building: 81.1s
Building: 81.2s
Building: 81.3s
Building: 81.4s
Building: 81.5s
Building: 81.6s
Building: 81.7s
Building: 81.8s
Building: 81.9s
Building: 82.0s
Building: 82.1s
Building: 82.2s
Building: 82.3s
Building: 82.4s
Building: 82.5s
Building: 82.6s
Building: 82.7s
Building: 82.8s
Building: 82.9s
Building: 83.0s
Building: 83.1s
Building: 83.2s
Building: 83.3s
Building: 83.4s
Building: 83.5s
Building: 83.7s
Building: 83.8s
Building: 83.9s
Building: 84.0s
Building: 84.1s
Building: 84.2s
Building: 84.3s
Building: 84.4s
Building: 84.5s
Building: 84.6s
Building

Building: 103.9s
Building: 104.0s
Building: 104.1s
Building: 104.2s
Building: 104.3s
Building: 104.4s
Building: 104.5s
Building: 104.6s
Building: 104.7s
Building: 104.8s
Building: 104.9s
Building: 105.0s
Building: 105.1s
Building: 105.2s
Building: 105.3s
Building: 105.4s
Building: 105.5s
Building: 105.7s
Building: 105.8s
Building: 105.9s
Building: 106.0s
Building: 106.1s
Building: 106.2s
Building: 106.3s
Building: 106.4s
Building: 106.5s
Building: 106.6s
Building: 106.7s
Building: 106.8s
Building: 106.9s
Building: 107.0s
Building: 107.1s
Building: 107.2s
Building: 107.3s
Building: 107.4s
Building: 107.5s
Building: 107.7s
Building: 107.8s
Building: 107.9s
Building: 108.0s
Building: 108.1s
Building: 108.2s
Building: 108.3s
Building: 108.4s
Building: 108.5s
Building: 108.6s
Building: 108.7s
Building: 108.8s
Building: 108.9s
Building: 109.0s
Building: 109.1s
Building: 109.2s
Building: 109.3s
Building: 109.4s
Building: 109.5s
Building: 109.6s
Building: 109.7s
Building: 109.8s
Building: 109.

...

Building: 204.1s
Building: 204.2s
Building: 204.3s
Building: 204.4s
Building: 204.5s
Building: 204.6s
Building: 204.7s
Building: 204.8s
Building: 204.9s
Building: 205.0s
Building: 205.1s
Building: 205.2s
Building: 205.3s
Building: 205.4s
Building: 205.5s
Building: 205.6s
Building: 205.7s
Building: 205.8s
Building: 205.9s
Building: 206.0s
Building: 206.1s
Building: 206.2s
Building: 206.3s
Building: 206.4s
Building: 206.5s
Building: 206.6s
Building: 206.7s
Building: 206.8s
Building: 207.0s
Building: 207.1s
Building: 207.2s
Building: 207.3s
Building: 207.4s
Building: 207.5s
Building: 207.6s
Building: 207.7s
Building: 207.8s
Building: 207.9s
Building: 208.0s
Building: 208.1s
Building: 208.2s
Building: 208.3s
Building: 208.4s
Building: 208.5s
Building: 208.6s
Building: 208.7s
Building: 208.8s
Building: 208.9s
Building: 209.0s
Building: 209.1s
Building: 209.2s
Building: 209.3s
Building: 209.4s
Building: 209.5s
Building: 209.6s
Building: 209.7s
Building: 209.8s
Building: 209.9s
Building: 210.

Building: 229.1s
Building: 229.2s
Building: 229.3s
Building: 229.4s
Building: 229.5s
Building: 229.6s
Building: 229.7s
Building: 229.8s
Building: 229.9s
Building: 230.0s
Building: 230.1s
Building: 230.2s
Building: 230.4s
Building: 230.5s
Building: 230.6s
Building: 230.7s
Building: 230.8s
Building: 230.9s
Building: 231.0s
Building: 231.1s
Building: 231.2s
Building: 231.3s
Building: 231.4s
Building: 231.5s
Building: 231.6s
Building: 231.7s
Building: 231.8s
Building: 231.9s
Building: 232.0s
Building: 232.1s
Building: 232.2s
Building: 232.3s
Building: 232.4s
Building: 232.6s
Building: 232.7s
Building: 232.8s
Building: 232.9s
Building: 233.0s
Building: 233.1s
Building: 233.2s
Building: 233.3s
Building: 233.4s
Building: 233.5s
Building: 233.6s
Building: 233.7s
Building: 233.8s
Building: 233.9s
Building: 234.0s
Building: 234.1s
Building: 234.2s
Building: 234.3s
Building: 234.4s
Building: 234.5s
Building: 234.6s
Building: 234.7s
Building: 234.8s
Building: 235.0s
Building: 235.1s
Building: 235.

Building: 254.2s
Building: 254.3s
Building: 254.4s
Building: 254.5s
Building: 254.6s
Building: 254.7s
Building: 254.8s
Building: 254.9s
Building: 255.0s
Building: 255.1s
Building: 255.2s
Building: 255.3s
Building: 255.4s
Building: 255.5s
Building: 255.6s
Building: 255.8s
Building: 255.9s
Building: 256.0s
Building: 256.1s
Building: 256.2s
Building: 256.3s
Building: 256.4s
Building: 256.5s
Building: 256.6s
Building: 256.7s
Building: 256.8s
Building: 256.9s
Building: 257.0s
Building: 257.1s
Building: 257.2s
Building: 257.3s
Building: 257.4s
Building: 257.5s
Building: 257.6s
Building: 257.7s
Building: 257.8s
Building: 257.9s
Building: 258.0s
Building: 258.1s
Building: 258.2s
Building: 258.3s
Building: 258.4s
Building: 258.5s
Building: 258.6s
Building: 258.8s
Building: 258.9s
Building: 259.0s
Building: 259.1s
Building: 259.2s
Building: 259.3s
Building: 259.4s
Building: 259.5s
Building: 259.6s
Building: 259.7s
Building: 259.8s
Building: 259.9s
Building: 260.0s
Building: 260.1s
Building: 260.

Building: 279.2s
Building: 279.3s
Building: 279.4s
Building: 279.5s
Building: 279.6s
Building: 279.8s
Building: 279.9s
Building: 280.0s
Building: 280.1s
Building: 280.2s
Building: 280.3s
Building: 280.4s
Building: 280.5s
Building: 280.6s
Building: 280.7s
Building: 280.8s
Building: 280.9s
Building: 281.0s
Building: 281.1s
Building: 281.2s
Building: 281.3s
Building: 281.4s
Building: 281.5s
Building: 281.6s
Building: 281.7s
Building: 281.8s
Building: 282.0s
Building: 282.1s
Building: 282.2s
Building: 282.3s
Building: 282.4s
Building: 282.5s
Building: 282.6s
Building: 282.7s
Building: 282.8s
Building: 282.9s
Building: 283.0s
Building: 283.1s
Building: 283.2s
Building: 283.3s
Building: 283.4s
Building: 283.5s
Building: 283.6s
Building: 283.7s
Building: 283.8s
Building: 283.9s
Building: 284.0s
Building: 284.1s
Building: 284.3s
Building: 284.4s
Building: 284.5s
Building: 284.6s
Building: 284.7s
Building: 284.8s
Building: 284.9s
Building: 285.0s
Building: 285.1s
Building: 285.2s
Building: 285.

TimeoutError: 

In [ ]:
class HierPcaSymmetricModel(HierarchicalSymmetricModel):
    
    def __init__(self, n_pca=8, beta_prior_std=1.0, intra_group_std=0.1, num_chains=4, num_samples=100):
        self.beta_prior_std = float(beta_prior_std)
        self.intra_group_std = float(intra_group_std)
        self.code = hier_code
        self.scale_ = None
        self.n_pca = n_pca
        self.pca = PCA(n_components=n_pca, whiten=True)
        self.fit = None
        self.num_chains = num_chains
        self.num_samples = num_samples
        
    def fit_predict(self, train_df, test_df, feat_cols):
        scale_ = (train_df[feat_cols]**2).mean(0)
        self.scale_ = scale_
        X_train = train_df[feat_cols] / scale_
        X_test = test_df[feat_cols] / scale_
        
        X_pca_train = self.pca.fit_transform(X_train)
        X_pca_test = self.pca.transform(X_test)

        y_train = train_df["targetWin"]
        y_test = test_df["targetWin"]

        ml_train = logit(train_df["p_fighter_implied"])
        ml_test = logit(test_df["p_fighter_implied"])
        
        is_m_train = train_df["gender"].map({"M":1, "W":0})
        is_m_test = test_df["gender"].map({"M":1, "W":0})
        
        data = {
            "n": train_df.shape[0],
            "n2": test_df.shape[0],
            "d": self.n_pca,
            "y": y_train.astype(int).values,
            "beta_prior_std": self.beta_prior_std,
            "intra_group_std": self.intra_group_std,
            "is_m": is_m_train.values,
            "is_m2": is_m_test.values,
            "X": X_pca_train,
            "ml_logit": ml_train.values,
            "X2": X_pca_test,
            "ml_logit2": ml_test.values,
        }
        fit = self._fit(data)
        return fit["y_pred"].mean(1)

hier_model = HierPcaSymmetricModel(n_pca=16, beta_prior_std=1.0, intra_group_std=0.1, num_samples=200)
y_pred = hier_model.fit_predict(train_gender_df, test_gender_df, feat_cols)
acc = (y_pred.round() == test_df["targetWin"]).mean()
print(f"accuracy: {acc}")
print(f"log loss: {log_loss(y_pred=y_pred, y_true=y_test)}")